# block encoding

block encoding 有多种实现方式：

1. 简单构造; [4]

2. Linear combination of unitaries; [1,3]

3. & 4. efficient method for sparse matrices and structured matrices. [2,4,5]

本部分参考文献：

[1]. https://pennylane.ai/qml/demos/tutorial_lcu_blockencoding/

[2]. https://pennylane.ai/qml/demos/tutorial_block_encoding/#fable

[3]. Hamiltonian Simulation by Qubitization, https://doi.org/10.22331/q-2019-07-12-163

[4]. EXPLICIT QUANTUM CIRCUITS FOR BLOCK ENCODINGS OF CERTAIN SPARSE MATRICES, https://doi.org/10.1137/22M1484298

[5]. FABLE: Fast Approximate Quantum Circuits for Block-Encodings, 
https://doi.org/10.1109/QCE53715.2022.00029

# 最简单的构造方式

\begin{equation}
U(A)=\left(\begin{array}{cc}A & \sqrt{I-A A^{\dagger}} \\ \sqrt{I-A^{\dagger} A} & -A^{\dagger}\end{array}\right).
\end{equation}

不要求矩阵A是厄米的

要求矩阵A的二范数小于等于1

对应的量子线路可能较深


In [5]:
import numpy as np
import scipy

In [6]:
I = np.array([[1,0],[0,1]])

A = np.array([[0.1, 0.2], [0.3, 0.4]]) # 测试矩阵1
# A = np.array([[7, -1], [-1, 3]]) # 测试矩阵4
# A = np.array([[5, 0], [0, 5]]) # 测试矩阵4
# A = np.array([[1, 2], [3, 4]]) # 测试矩阵2
# A = np.array([[0.5, -0.9], [0.8, 0.5]]) # 测试矩阵3
assert np.linalg.norm(A, ord=2) <= 1, Exception(f'矩阵的二范数为{np.linalg.norm(A, ord=2)}，大于1')

B = scipy.linalg.sqrtm(I - A @ A.transpose().conj())
C = scipy.linalg.sqrtm(I - A.transpose().conj() @ A)
D = -A.transpose().conj()
AB = np.concatenate((A, B), axis=1)
CD = np.concatenate((C, D), axis=1)
U = np.concatenate((AB, CD), axis=0)
print(A)
print(U[:2,:2])
print((U.transpose().conj() @ U).round(10)) # 验证幺正

[[0.1 0.2]
 [0.3 0.4]]
[[0.1 0.2]
 [0.3 0.4]]
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1. -0.]
 [ 0.  0. -0.  1.]]


In [7]:
A, B, AB

(array([[0.1, 0.2],
        [0.3, 0.4]]),
 array([[ 0.97283788, -0.05988708],
        [-0.05988708,  0.86395228]]),
 array([[ 0.1       ,  0.2       ,  0.97283788, -0.05988708],
        [ 0.3       ,  0.4       , -0.05988708,  0.86395228]]))

# Linear combination of unitaries (LCU)

矩阵A可以分解为N个幺正矩阵的线性叠加

\begin{equation}
A=\sum_{k=0}^{N-1} \alpha_k U_k .
\end{equation}

幺正矩阵的选取有很多种方式，最简单的是选择Pauli基。分解代码如下

In [15]:
import math
import itertools
from functools import reduce
NP_DTYPE = np.complex64
sigmaI = np.eye(2, dtype=NP_DTYPE)
sigmaX = np.array([[0, 1], [1, 0]], dtype=NP_DTYPE)
sigmaY = np.array([[0, -1j], [1j, 0]], dtype=NP_DTYPE)
sigmaZ = np.array([[1, 0], [0, -1]], dtype=NP_DTYPE)

def pauli_decompose(A: np.ndarray) -> dict:
    '''
    将矩阵A分解为Pauli基的叠加
    '''
    q_num = int(math.log2(A.shape[0]))
    if q_num >= 14:
        raise Exception('qubit number too large, memory consumption too large')
    Pauli_basis = np.asarray([sigmaI, sigmaX, sigmaY, sigmaZ])
    A = A.reshape([2] * q_num * 2)
    A = np.transpose(A, np.hstack([[i, i + q_num] for i in range(q_num)]))
    for _ in range(q_num):
        A = np.einsum('abc,dba->cd', A.reshape([2, 2, -1]), Pauli_basis)
    return {
        ''.join(k): v
        for k, v in zip(list(itertools.product('IXYZ', repeat=q_num)),
                        A.reshape(-1) / 2**q_num)
    }

def recombination(pauli_coe: dict):
    '''
    验证重组后能得到A矩阵
    '''
    op_map = {'I': sigmaI, 'X': sigmaX, 'Y': sigmaY, 'Z': sigmaZ}
    q_num = len(list(pauli_coe.keys())[0])
    # assert len(pauli_coe) == 4**q_num
    rho = 0
    for op, expctation in pauli_coe.items():
        rho += expctation * reduce(np.kron, [op_map[_op] for _op in op])
    return rho

In [18]:
# 测试
# A = np.array([[0.1, 0.2], [0.3, 0.4]]) # 测试矩阵1
# A = np.array([[1, 2], [3, 4]]) # 测试矩阵2
A = np.array([[0.5, -0.8], [-0.8, 0.5]]) # 测试矩阵3
# A = np.array([[0.5, -0.5], [-0.5, 0.5]]) # 测试矩阵4
# A = np.array([[0.5, -0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5], [0.5, -0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5]]) # 测试矩阵5

pauli_coe = pauli_decompose(A)
print(pauli_coe)
print(A)
print(recombination(pauli_coe))

{'I': (0.5+0j), 'X': (-0.8+0j), 'Y': 0j, 'Z': 0j}
[[ 0.5 -0.8]
 [-0.8  0.5]]
[[ 0.5+0.j -0.8+0.j]
 [-0.8+0.j  0.5+0.j]]


In [19]:
# 对比pennylane的分解代码
import pennylane as qml
LCU = qml.pauli_decompose(A)
LCU_coeffs, LCU_ops = LCU.terms()

In [20]:
LCU_coeffs

array([ 0.5, -0.8])

In [21]:
LCU_ops

[I(0), X(0)]

得到了分解后Pauli基的系数后，就可以开始进行block encoding，过程中要用到prepare（PREP）算符和select（SEL）算符：

\begin{equation}
\operatorname{PREP}|0\rangle=\sum_k \sqrt{\frac{\left|\alpha_k\right|}{\lambda}}|k\rangle,
\end{equation}

其中$\lambda$是归一化常数，$\lambda=\sum_k\left|\alpha_k\right|$

\begin{equation}
\text { SEL }|k\rangle|\psi\rangle=|k\rangle U_k|\psi\rangle \text {. }
\end{equation}

A矩阵的block encoding就是

\begin{equation}
U=\text { PREP }^{\dagger} \cdot \text { SEL } \cdot \text { PREP }
\end{equation}

\begin{equation}
U=\left[
\begin{array}{ll}
A & * \\
* & *
\end{array}
\right]
\end{equation}

这一点可以通过如下运算来验证

\begin{align}
\left\langle 0\left|\mathrm{PREP}^{\dagger} \cdot \mathrm{SEL} \cdot \mathrm{PREP}\right| 0\right\rangle|\psi\rangle 
&=\sum_k \left\langle k \left| \sqrt{\frac{\left|\alpha_k\right|}{\lambda}} \cdot \sqrt{\frac{\left|\alpha_k\right|}{\lambda}} \right| k \right \rangle U_k \left| \psi \right\rangle \\
&=\sum_k \frac{\left|\alpha_k\right|}{\lambda} U_k |\psi\rangle\\
&=\frac{A}{\lambda}|\psi\rangle \\
\end{align}

注意原本分解系数$\alpha_k$是个复数，但上式告诉我们要将分解系数化为非负实数，这可以通过将一个相位吸收到$U_k$中做到。

In [13]:
import qiskit
from qiskit_aer import AerSimulator

In [23]:
basis2unitary = {'I': sigmaI, 'X': sigmaX, 'Y': sigmaY, 'Z': sigmaZ}

# A = np.array([[0.1, 0.2], [0.3, 0.4]]) # 测试矩阵1
# A = np.array([[1, 2], [3, 4]]) # 测试矩阵2
A = np.array([[0.5, -0.8], [-0.8, 0.5]]) # 测试矩阵3
# A = np.array([[0.5, -0.5], [-0.5, 0.5]]) # 测试矩阵4
# A = np.array([[0.9, -0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5], [0.5, -0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5]]) # 测试矩阵5

def judge_2_powers(k: int):
    '''
    判断一个数字是否为2的整数幂
    返回 True/False 及大于等于k的最小的2的幂次
    '''
    bitstring = np.binary_repr(k)
    if bitstring == '0':
        return False, 0
    elif bitstring == '1':
        return False, 1
    elif bitstring[0] == '1' and int(bitstring[1:]) == 0:
        return True, len(bitstring) - 1
    else:
        return False, len(bitstring)


def block_encoding_LCU_pauli(A: np.array):
    # TODO：如果A的维度不是2的整数幂，需要填充0补齐
    # A = padded(A)

    qnum_encode = judge_2_powers(A.shape[0])[-1] # A矩阵占据的维度
    qnum_ancilla = 2 * qnum_encode # 编码系数的辅助比特，Pauli基数目随比特数N成4^N增长
    qnum_all = qnum_encode + qnum_ancilla

    alphas = pauli_decompose(A)
    init_state = np.abs(list(alphas.values()))
    Lambda = np.sum(init_state)
    init_state = np.sqrt(init_state/Lambda)
    qc_init = qiskit.QuantumCircuit(qnum_ancilla)
    qc_init.initialize(init_state, list(np.arange(qnum_ancilla)))
    qc_init = qiskit.transpile(qc_init, basis_gates=['u', 'cz'], optimization_level=3)

    qc_all = qiskit.QuantumCircuit(qnum_all)
    new_qc_init = qiskit.QuantumCircuit(qnum_ancilla)
    ## delete the reset gate in the initialization circuit
    for inst, qargs, cargs in qc_init.data:
        if inst.name == 'reset':
            continue
        new_qc_init.append(inst, qargs, cargs)
    # print(new_qc_init.draw())

    qc_all.compose(new_qc_init, range(qnum_ancilla), inplace=True)

    for idx1, pauli_basis in enumerate(alphas.keys()):
        coe = alphas[pauli_basis]
        if np.abs(coe) == 0:
            continue
        for idx2, _pauli_basis in enumerate(pauli_basis):
            qc_mcg = qiskit.QuantumCircuit(1) # 构建多比特控制门
            # qc_mcg.rz(-2*np.angle(coe) * (idx2 == 0),[0])
            if idx2 == 0:
                qc_mcg.u(0, 0, np.angle(coe), [0])
                qc_mcg.x([0])
                qc_mcg.u(0, 0, np.angle(coe), [0])
                qc_mcg.x([0])
            # for idx2, _pauli_basis in enumerate(pauli_basis):
            #     qc_mcg = qiskit.QuantumCircuit(1) # 构建多比特控制门
            #     qc_mcg.unitary(np.exp(1j*np.angle(coe) * (idx2 == 0)) * basis2unitary[_pauli_basis], [0])
            #     mcg = qc_mcg.to_gate().control(qnum_ancilla, ctrl_state=np.binary_repr(idx1, width=qnum_ancilla))
            #     qc_all.append(mcg, list(range(qnum_ancilla)) + [qnum_ancilla+idx2])
            if _pauli_basis == 'X':
                # qc_mcg.rx(-2*np.angle(coe) * (idx2 == 0), [0])
                qc_mcg.x([0])
            elif _pauli_basis == 'Y':
                # qc_mcg.ry(-2*np.angle(coe) * (idx2 == 0), [0])
                qc_mcg.y([0])
            elif _pauli_basis == 'Z':
                # qc_mcg.rz(-2*np.angle(coe) * (idx2 == 0), [0])
                qc_mcg.z([0])
            else:
                # qc_mcg.p(-2*np.angle(coe),[0])
                qc_mcg.id([0])
            # if idx2 == 0:
                
            mcg = qc_mcg.to_gate().control(qnum_ancilla, ctrl_state=np.binary_repr(idx1, width=qnum_ancilla))
            qc_all.append(mcg, list(range(qnum_ancilla)) + [qnum_ancilla+idx2])

    qc_all.compose(new_qc_init.inverse(), range(qnum_ancilla), inplace=True)
    

    return qc_all, Lambda

qc, Lambda = block_encoding_LCU_pauli(A)
print(qc.draw())
simulator = AerSimulator(method='unitary')
qc = qiskit.transpile(qc, simulator, basis_gates=['u', 'cz'], optimization_level=3)
qc.save_unitary()
result = simulator.run(qc.reverse_bits()).result()
print(A/Lambda)
print(result.get_unitary().data.round(10)[:A.shape[0],:A.shape[1]])

     ┌─────────────────┐   ┌──────────────────┐   ┌────────────┐               »
q_0: ┤ U(0.66896,0,-π) ├─■─┤ U(0.90183,-π,-π) ├─■─┤ U(π/2,0,π) ├───────o───────»
     └─────────────────┘ │ └──────────────────┘ │ └────────────┘       │       »
q_1: ────────────────────■──────────────────────■──────────────────────o───────»
                                                                ┌──────┴──────┐»
q_2: ───────────────────────────────────────────────────────────┤ circuit-484 ├»
                                                                └─────────────┘»
«                    ┌──────────────┐   ┌─────────────────┐   »
«q_0: ───────■───────┤ U(-π/2,-π,0) ├─■─┤ U(-0.90183,π,π) ├─■─»
«            │       └──────────────┘ │ └─────────────────┘ │ »
«q_1: ───────o────────────────────────■─────────────────────■─»
«     ┌──────┴──────┐                                         »
«q_2: ┤ circuit-495 ├─────────────────────────────────────────»
«     └─────────────┘                            